# Text als Sprache

Als *Natural Language Processing* bezeichnet man die computergestützte Verarbeitung von natürlichen Sprachen, also von von Menschen gesprochenen Sprachen im Gegensatz zu Programmiersprachen. Computer sind gut darin, Programmiersprachen mit ihren starren Regeln zu verstehen, aber menschliche Sprachen mit ihren Unregelmäßigkeiten stellen immer noch eine Herausforderung dar. Die Wissenschaft, die sich mit diesen Problemen beschäftigt, ist die Computerlinguistik.

Methoden der Computerlinguistik sind mittlerweile für eine Vielzahl von Anwendungsbereichen zentral. So sind etwa Suchmaschinen darauf trainiert, nicht nur den exakten Text einer Suchanfrage zu finden, sondern auch andere Flexionsformen. Eine Google-Suche nach „Parties in Hamburg“ findet so z.B. auch die Seite [Nachtleben & Party – hamburg.de](http://www.hamburg.de/nachtleben-party/). Dazu muss die Suchmaschine wissen, dass „Party“ der Singular von „Parties“ ist, und dass „in“ als Präposition für den gesuchten Inhalt nicht zwingend relevant ist.

Der Ausgangspunkt für NLP ist dabei immer der reine Text, ohne Formatierung oder ähnliches. Solche Dateien können in Python problemlos eingelesen werden. Dabei muss allerdings die [Zeichencodierung](http://de.wikipedia.org/wiki/Zeichenkodierung) des Textes bekannt sein. In der Regel arbeitet man heutzutage mit [UTF-8](http://de.wikipedia.org/wiki/UTF-8).

Als Beispiel soll der Anfang einer Rede der deutschen Bundeskanzlerin Angela Merkel dienen. (Der Anschaulichkeit halber ist die Anrede entfernt.) Um die Datei `Rede_Rundfunk.txt` mit der Kodierung `UTF-8` zum Lesen (`r`) zu öffnen, dient dieser Befehl:

In [1]:
import codecs
with codecs.open('Rede_Rundfunk.txt', 'r', 'utf-8') as infile:
    rede = infile.read()
print rede[0:100]

vor 30 Jahren waren Sie einer der Geburtshelfer des privaten Rundfunks. Ihre Worte von damals sind l


Wenn wir mit Texten arbeiten, sind wir meist aber nicht am Text als Ganzem interessiert, sondern an den Wörtern, aus denen er besteht. Wörter sind in den europäischen Sprachen meist durch Leerzeichen getrennt. So sehen die ersten zwanzig Wörter aus, wenn man den Text entsprechend aufspaltet:

In [2]:
print ' -- '.join(rede.split()[0:20])

vor -- 30 -- Jahren -- waren -- Sie -- einer -- der -- Geburtshelfer -- des -- privaten -- Rundfunks. -- Ihre -- Worte -- von -- damals -- sind -- legendär. -- Deshalb -- möchte -- ich


Hier sieht man schon einige Probleme. Insbesondere werden hier die Satzzeichen noch beibehalten. Die verfälschen aber das Ergebnis, denn das Wort heißt ja nicht „legendär**.**“, sondern einfach „legendär“. Schon bei so einfachen Aufgaben wie dem Aufspalten kann es also nützlich sein, etwas ausgefeiltere Werkzeuge heranzuziehen. Einen einfachen Einstieg bietet dabei das Modul `TextBlob`, für das es auch eine auf die deutsche Sprache ausgelegte Version gibt:

In [3]:
from textblob_de import TextBlobDE as TextBlob
blob = TextBlob(rede)

In [4]:
print u' -- '.join(blob.words[0:20])

vor -- 30 -- Jahren -- waren -- Sie -- einer -- der -- Geburtshelfer -- des -- privaten -- Rundfunks -- Ihre -- Worte -- von -- damals -- sind -- legendär -- Deshalb -- möchte -- ich


`TextBlob` stellt noch eine Reihe weiterer Methoden aus der Sprachverarbeitung bereit. So kann man etwa Worte auf ihre Grundformen zurückführen. Gerade bei stark flektierenden Sprachen wie dem Deutschen ist das oft nützlich. Diesen Schritt nennt man in der Computerlinguistik Lemmatisierung.

In [5]:
print u' -- '.join(blob.words[0:20].lemmatize())

vor -- 30 -- Jahren -- sein -- Sie -- ein -- der -- Geburtshelfer -- des -- privat -- Rundfunks -- Ihre -- Wort -- von -- damals -- sein -- legendär -- Deshalb -- mögen -- ich


Man kann sehen, dass der Schritt für Verben gut funktioniert. So ist aus „waren“ „sein“ geworden und aus „möchte“ „mögen“. Bei Substantiven scheint dieser Algorithmus jedoch an seine Grenzen zu kommen: „Jahren“ ist nicht zu „Jahr“ geworden und „Rundfunks“ nicht zu „Rundfunk“. In Anwendungsfällen, in denen eine hohe Qualität der Verarbeitung wichtig ist, muss man also ggf. nach besseren Werkzeugen suchen oder die vorhandenen verbessern.

Für speziellere Anwendungen kann man auch die Grammatik eines Texts analysieren. Ein grundlegender Schritt ist dabei die Bestimming der Wortart (Substantiv, Verb, etc.), englisch „Part of Speech“. Hierfür werden in der Linguistik meist bestimmte Kürzel verwendet, die von `TextBlob` sind auf [dieser Seite](http://www.clips.ua.ac.be/pages/mbsp-tags) aufgelistet.

In [6]:
print u' -- '.join([u'{}/{}'.format(word, tag) for word, tag in blob.pos_tags[0:20]])

vor/IN -- 30/CD -- Jahren/NN -- waren/VB -- Sie/PRP -- einer/DT -- der/DT -- Geburtshelfer/NN -- des/DT -- privaten/JJ -- Rundfunks/NN -- Ihre/PRP$ -- Worte/NNS -- von/IN -- damals/RB -- sind/VB -- legendär/JJ -- Deshalb/RB -- möchte/VB -- ich/PRP


Dies kann etwa dazu verwendet werden, für bestimmte Anwendungen nur bestimmte Wortarten zu berücksichtigen. Falls etwa nur die Substantive interessieren, lassen sie sich aufgrund der PoS-Information herausfiltern.

Um Informationen wie Lemma und Wortart nicht einzeln erzeugen zu müssen, kann man diese mit der `parse()`-Funktion auch in einem Durchlauf erzeugen. Da der Standard-Parser keine Lemmata erzeugt, müssen diese zunächst explizit aktiviert werden.

In [7]:
from textblob_de import PatternParser
blob = TextBlob(rede, parser=PatternParser(lemmata=True))
parse = blob.parse()
parse[0:100]

u'vor/IN/B-PP/B-PNP/vor 30/CD/B-NP/I-PNP/30 Jahren/NN/I-NP/I-PNP/jahren waren/VB/B-VP/O/sein Sie/PRP/B'

Wie man sieht, werden hier alle Informationen zu einem Wort durch Schrägstriche getrennt angegeben. Dies ist eine übliche Konvention in der Linguistik. Die Bedeutung der einzelnen Tags kann man sich anzeigen lassen:

In [8]:
parse.tags

[u'word', u'part-of-speech', u'chunk', u'preposition', u'lemma']

In dieser Form sind die Angaben nicht sehr leicht zu entziffern. Man kann sie aber leicht in eine besser zu verarbeitende Form überführen. In Python steht dafür `namedtuple()` zur Verfügung, mit dem man einzelne Felder in einer Liste (nichts anderes ist ein sogenanntes „tuple“ im Grunde) benennen kann.

Da Feldnamen in Python kein '-' enthalten dürfen, wird es zunächst durch einen Unterstrich ersetzt. Auf der Grundlage können wir eine neue Datenstruktur (Klasse) erzeugen, die einen einfachen Zugriff auf die einzelnen Informationen erlaubt.

In [9]:
from collections import namedtuple
fieldnames = [tag.replace('-', '_') for tag in parse.tags]
Token = namedtuple('Token', fieldnames)
Token('vor', 'IN', 'B-PP', 'B-PNP', 'vor')

Token(word='vor', part_of_speech='IN', chunk='B-PP', preposition='B-PNP', lemma='vor')

Der Klasse `Token` müssen die Informationen als einzelne Argumente übergeben werden. Das ist dann ein Problem, wenn die Informationen als Liste in einer einzelnen Variable vorliegen.

In [10]:
fields = ['vor', 'IN', 'B-PP', 'B-PNP', 'vor']
Token(fields)

TypeError: __new__() takes exactly 6 arguments (2 given)

Die Klasse hätte fünf Argumente erwartet, hat aber nur eines, nämlich die Liste mit fünf Elementen, erhalten. (Ein Argument wird immer intern verwendet, daher spricht die Fehlermeldung von 6 und 2 statt von 5 und 1.) Mit `*` lassen sich aber Listen statt einzelner Argumente übergeben.

In [11]:
Token(*fields)

Token(word='vor', part_of_speech='IN', chunk='B-PP', preposition='B-PNP', lemma='vor')

Dies kann man sich nun zunutze machen, um aus dem „Parse“ des Textes eine besser zu verarbeitende Datenstruktur zu gewinnen. Da einzelne Worte durch Leerzeichen und die einzelnen Informationen pro Wort durch Schrägstriche getrennt sind, muss der Parse nur zweimal aufgespalten werden. Die dadurch gewonnenen Einzelinformationen pro Wort werden in der Token-Klasse gespeichert.

In [12]:
tokens = [Token(*token.split('/')) for token in parse.split(' ')]
tokens[0:5]

[Token(word=u'vor', part_of_speech=u'IN', chunk=u'B-PP', preposition=u'B-PNP', lemma=u'vor'),
 Token(word=u'30', part_of_speech=u'CD', chunk=u'B-NP', preposition=u'I-PNP', lemma=u'30'),
 Token(word=u'Jahren', part_of_speech=u'NN', chunk=u'I-NP', preposition=u'I-PNP', lemma=u'jahren'),
 Token(word=u'waren', part_of_speech=u'VB', chunk=u'B-VP', preposition=u'O', lemma=u'sein'),
 Token(word=u'Sie', part_of_speech=u'PRP', chunk=u'B-NP', preposition=u'O', lemma=u'sie')]

In [13]:
def lemmatize_and_filter(tokens, tags):
    result = []
    for token in tokens:
        pos = token.part_of_speech[0:2]
        if pos in tags:
            if pos == 'NN':
                # Substantive immer groß schreiben
                result.append(token.lemma.title())
            else:
                restult.append(token.lemma)
    return result

print u' -- '.join(lemmatize_and_filter(tokens, ['NN'])[0:20])

Jahren -- Geburtshelfer -- Rundfunks -- Wort -- Kellerstudio -- Ludwigshafen -- Zuschauer -- Morgen -- Januar -- Worten -- „Meine -- Damen -- Herren -- Moment -- Zeug -- Starts -- Fernsehveranstalters -- Bundesrepublik -- Deutschland -- Zeugen


Das Verfahren funktioniert in weiten Teilen, aber alle Methoden der Computerlinguistik sind mit einer gewissen Fehlerrate behaftet. Gute Verfahren haben dabei eine Genauigkeit von über 90% – was in der Gesamtschau immer noch eine Menge Fehler sind. In diesem Fall sieht man, dass der Wortfinde-Algorithmus nicht mit den deutschen Anführungszeichen umgehen kann. Hier würde eine entsprechende Vorbereitung des Textes helfen.

Ein anderer Ansatz kann sein, nicht über die Wortarten, sondern über eine vorgegebene Liste an Wörtern zu filtern. Je nach Anwendungszweck kann eine solche Stoppwortliste länger oder kürzer sein. Ihr Ziel ist es, nicht bedeutungstragende Wörter herauszufiltern. Dies können neben Partikeln auch Hilfsverben und unspezifische Adjektive und Adverben (z.B. „sehr“) sein.

In [14]:
with codecs.open('stopwords.txt', 'r', 'utf-8') as stopwordfile:
    stopwords_de = [line.strip() for line in stopwordfile.readlines()]

from string import punctuation

def lemmatize_and_filter2(tokens, stopwords):
    result = []
    for token in tokens:
        pos = token.part_of_speech[0:2]
        word = token.word
        if not word.lower() in stopwords and not word.isdigit() and not word in punctuation:
            if pos == 'NN':
                result.append(token.lemma.title())
            else:
                result.append(token.lemma)
    return result

print u' -- '.join(lemmatize_and_filter2(tokens, stopwords_de)[0:20])

Jahren -- Geburtshelfer -- privat -- Rundfunks -- Wort -- legendär -- zitieren -- Kellerstudio -- Ludwigshafen -- begrüßen -- Zuschauer -- Januar -- Worten -- „Meine -- verehrt -- Damen -- Herren -- Moment -- Zeug -- Starts
